In [5]:
import os

In [6]:
%pwd

'e:\\Kidney Disease Classification\\Kidney_Disease_Classification_END_to_END\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'e:\\Kidney Disease Classification\\Kidney_Disease_Classification_END_to_END'

In [9]:
#UPDATING ENTITY
from dataclasses import dataclass #SIMPLIFIES THE CREATION OF CLASSES THAT STORES DATA
from pathlib import Path

#THIS DECORATOR AUTOMATICALLY GENERATES SPECIAL METHODS FOR THE CLASS
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
#UPDATING THE CONFIGURATION MANGGER IN SRC CONFIG
class ConfigurationManager:
    #DEFINING A SPECIAL FUNCTION FOR INITIALIZING OBECTS
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [13]:
#UPDATING THE COMPONENTS
class DataIngestion:
    #DEFINING A SPECIAL FUNCTION FOR INITIALIZING OBECTS
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    #WHILE DEFINING A FUNCTION IN A CLASS WE TAKE 'SELF' AS INSTANCE PARAMETER
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
#UPDATING THE PIPELINE
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-12-21 22:48:26,069:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-12-21 22:48:26,088:INFO:common:yaml file: params.yaml loaded successfully]
[2023-12-21 22:48:26,091:INFO:common:created directory at: artifacts]
[2023-12-21 22:48:26,191:INFO:common:created directory at: artifacts/data_ingestion]
[2023-12-21 22:48:26,195:INFO:3636760838:Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=404c2dc3-7dca-4586-9904-396969aed66a
To: e:\Kidney Disease Classification\Kidney_Disease_Classification_END_to_END\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:15<00:00, 3.74MB/s]

[2023-12-21 22:48:45,415:INFO:3636760838:Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
